In [1]:
# Mengimpor library yang diperlukan
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
 
# Mengimpor dataset
dataset_train = pd.read_excel('FB_training.xlsx')
training_set = dataset_train.iloc[:, 1:2].values
 
# Proses feature scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
 
# Membuat prediksi dengan 60 time-window (3 bulan)
X_train = []
y_train = []
for i in range(60, training_set.shape[0]):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
 
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
 
# Mengimpor library Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
 
# Mulai membuat RNN
Mesin_saham = Sequential()
 
# Menambah layer LSTM yang pertama dan Dropout regularisation
Mesin_saham.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
Mesin_saham.add(Dropout(0.2))
 
# Menambah layer LSTM yang kedua dan Dropout regularisation
Mesin_saham.add(LSTM(units = 50, return_sequences = True))
Mesin_saham.add(Dropout(0.2))
 
# Menambah layer LSTM yang ketiga dan Dropout regularisation
Mesin_saham.add(LSTM(units = 50, return_sequences = True))
Mesin_saham.add(Dropout(0.2))
 
# Menambah layer LSTM yang keempat dan Dropout regularisation
Mesin_saham.add(LSTM(units = 50))
Mesin_saham.add(Dropout(0.2))
 
# Menambahkan output layer
Mesin_saham.add(Dense(units = 1))
 
# Melihat rancangan network LSTM kita
Mesin_saham.summary()
 
# Compile RNN
Mesin_saham.compile(optimizer = 'adam', loss = 'mean_squared_error')
 
# Menjalankan RNN ke Training set
Mesin_saham.fit(X_train, y_train, epochs = 100, batch_size = 32)
 
# Mengimpor data saham sesungguhnya untuk Test set
dataset_test = pd.read_excel('FB_test.xlsx')
saham_real = dataset_test.iloc[:, 1:2].values
 
# Memprediksi harga saham
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, saham_real.shape[0] + X_train.shape[1]):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = Mesin_saham.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
 
# Visualisasi perbandingan hasil prediksi dan data sesunguhnya
plt.plot(saham_real, color = 'red', label = 'Harga saham Facebook sesungguhnya')
plt.plot(predicted_stock_price, color = 'blue', label = 'Harga saham Facebook prediksi')
plt.title('Prediksi harga saham Facebook')
plt.xlabel('Waktu')
plt.ylabel('Harga saham Facebook')
plt.legend()
plt.show()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 50)            10400     
_________________________________________________________________
dropout (Dropout)            (None, 60, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 60, 50)            20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 60, 50)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                2

KeyboardInterrupt: 